# Working with the parquet store

This is a brief demo that shows how to read the Parquet store.

## Set parameters

Let's make sure that this is the only place where users have to change contents.

In [1]:
# parameters
pq_data_dir = "/p/project/training2005/geomar_challenge/data/med_sea_connectivity_v2020.11.03.1/"

## Create Dask Cluster

In [2]:
import dask
from dask.distributed import Client, wait

# Make sure the Dask dashboard is easy to reach
dask.config.set(
    {
        'distributed.dashboard.link':
        "{JUPYTERHUB_BASE_URL}user/{JUPYTERHUB_USER}/{JUPYTERHUB_SERVER_NAME}/proxy/{port}/status"
    }
)

# start a Dask cluster that spans a whole node
client = Client(n_workers=2, threads_per_worker=4, memory_limit=120e9)
client

Client Scheduler: tcp://127.0.0.1:32866 Dashboard: /user/wrath@geomar.de/jupyterlab_1/proxy/8787/status,Cluster Workers: 2 Cores: 8 Memory: 240.00 GB


## Make sure the cluster works

In [4]:
from dask import array as darr

In [5]:
%%time

x = darr.random.normal(size=(100_000_000, 10), chunks=(100_000, 10))
display(x)
print(x.mean().compute())

dask.array<normal, shape=(100000000, 10), dtype=float64, chunksize=(100000, 10), chunktype=numpy.ndarray>

5.821575677452736e-05
CPU times: user 1.29 s, sys: 71.4 ms, total: 1.36 s
Wall time: 6.37 s


In [6]:
from dask import dataframe as ddf

In [7]:
%%time

dataframe = ddf.read_parquet(f"{pq_data_dir}/medsea-trajectories.pq")
dataframe

CPU times: user 165 ms, sys: 82.6 ms, total: 248 ms
Wall time: 2.08 s


,MPA,distance,land,lat,lon,temp,time,z,stokes,trajectory_id,step
npartitions=1582,,,,,,,,,,,
0,float32,float32,float32,float32,float32,float32,datetime64[ns],float32,bool,int64,int64
9620000,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
15207873200,...,...,...,...,...,...,...,...,...,...,...
15213144959,...,...,...,...,...,...,...,...,...,...,...


In [10]:
%time display(dataframe.loc[:5_000_000]["distance"].mean().compute())

208.0087519982496

CPU times: user 53.1 ms, sys: 8.73 ms, total: 61.8 ms
Wall time: 898 ms


There's a bug (?) in Dask dataframe that leads to high memory use when directly applying reductions to colums of Dask dataframes (not happening with the small subset used in the cell above). A workaround is to cast the column into a Dask array before reducing it:

In [11]:
%time display(dataframe.loc[:5_000_000]["distance"].to_dask_array().mean().compute())

208.00876

CPU times: user 52.8 ms, sys: 4.73 ms, total: 57.5 ms
Wall time: 870 ms


In [12]:
%time display(dataframe["distance"].to_dask_array().mean().compute())

160.74611

CPU times: user 22 s, sys: 1.63 s, total: 23.7 s
Wall time: 5min 23s
